In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
from datetime import datetime

In [2]:
ativo = 'BOVA11'
login = 0
password = ''
server = ''
path  = ''

In [3]:
mt5.initialize(path = path,
               login = login,
               password = password,
               server  =server
          )

True

In [4]:
#Função que agrupa um conjunto de dados por decil - 10%
def group_by_decil(train = None, test = None):
    d1  = np.percentile(train, q = 10)
    d2  = np.percentile(train, q = 20)
    d3  = np.percentile(train, q = 30)
    d4  = np.percentile(train, q = 40)
    d5  = np.percentile(train, q = 50)
    d6  = np.percentile(train, q = 60)
    d7  = np.percentile(train, q = 70)
    d8  = np.percentile(train, q = 80)
    d9  = np.percentile(train, q = 90)
    
    
    group = np.where(test <= d1, 1, 
                     np.where(test <= d2, 2, 
                     np.where(test <= d3, 3, 
                     np.where(test <= d4, 4, 
                     np.where(test <= d5, 5, 
                     np.where(test <= d6, 6, 
                     np.where(test <= d7, 7, 
                     np.where(test <= d8, 8,
                     np.where(test <= d9, 9, 10)))))))))
    return(group)
    

In [5]:
#Prepara Indicador Estocástico
def prepare_k_indicator (var   = None, 
                         var_h = None,
                         var_l = None,
                         df    = None, 
                         time  = 14):
    
    k_name    = 'K - ' + str(time) + ' - ' + var
    ks_name   = 'Ks - ' + str(time) + ' - ' + var
    size_name = 'K_Size - ' + str(time) + ' - ' + var
    
    name_high = 'High - ' + str(time) + ' - ' + var_h
    name_low  = 'High - ' + str(time) + ' - ' + var_l
    
    df[name_high] = df[var_h].rolling(time).max()
    df[name_low]  = df[var_l].rolling(time).min()
    
    df[size_name] = df[name_high] - df[name_low]
    
    df[k_name] = (df[var] - df[name_low])/df[size_name]
    
    df[k_name] = (df[k_name] * 100)/10
    
    df[ks_name] = df[k_name].rolling(3).mean()
    
    return(df)

In [6]:
#Retorna componentes da data
def select_dates(value):

    day   = value[:10]
    hour  = int(value[11:13])
    month = day[5:7]
    year  = day[:4]
    return([day,year ,month,hour])


In [7]:
#Retorna alvo de volume, tendo como base o acumulado das últimas 2h
def fix_data(df_2 = None):
    df_2['hour_2'] = np.where(df_2['hour'] <= 11, df_2['day'] +"_1", 
                          np.where(df_2['hour'] <= 13, df_2['day'] +"_2", 
                          np.where(df_2['hour'] <= 15, df_2['day'] +"_3", df_2['day'] +"_4")))

    dados_volume = df_2.groupby(['hour_2'])['real_volume'].sum().reset_index()
    
    dados_volume['volume_m20'] = dados_volume['real_volume'].rolling(20).mean()
    
    dados_volume['Previous_avg'] = dados_volume['volume_m20'].shift()
    
    dados_volume = dados_volume.drop('real_volume', axis = 1)
    
    df_2 = pd.merge(df_2, dados_volume, on = 'hour_2')
    
    df_2 = df_2.dropna().reset_index().iloc[:,1:]
    
    return(df_2)

In [8]:
#Agrupa os dados selecionados para compor a barra agrupada pelo volume
def group_bar(cj = None, final = 0, target_vol = 0):

    open_  = cj['open'].values.ravel()[0]
    close_ = cj['close'].values.ravel()[-1]
    high   = max(cj['high'].values)
    low    = min(cj['low'].values)
    total_vol     = cj['real_volume'].sum()
    total_dir_vol = cj['vol_dir'].sum()
    total_price_vol = cj['Preco_Vol'].sum()
    vwap = total_price_vol/total_vol
    open_time  = cj['time'].values.ravel()[0]
    close_time = cj['time'].values.ravel()[-1]
    
    barras = len(cj)
    
    final_data = [open_time,close_time,open_,high, low,close_, vwap, total_vol,total_dir_vol, target_vol, barras,final]

    return(final_data)

In [9]:
#Inicia variaveis basicas
def set_base_var(df = None):
    
    df['vol_dir'] = np.where(df['close'] > df['open'], df['real_volume'], 
                                np.where(df['close'] < df['open'], -df['real_volume'],0))

    df['Preco_Vol'] = df['close'] * df['real_volume']
    
    return(df.copy())
    

In [10]:
#Agrupa os dados até atingir a meta
def prepare_group(df_2 = None):

    continuar = True
    dados_agrupados = []
    vol_incr        = 0.1
    while(continuar):
      start_index = 0
      if(len(dados_agrupados) != 0):
        start_index = dados_agrupados[-1][-1] + 1
      if(start_index >= len(df_2)):
        continuar = False
        break
      target_vol = df_2['Previous_avg'].values.ravel()[start_index] * (1 + vol_incr)

      currenct_vol = 0
      final_index = 0
      currenct_index = start_index
      continuar_ = True
      while((currenct_vol < target_vol) and continuar == True):

        if(currenct_index >= len(df_2)):
          print('erro')
          continuar = False

          break

        line_vol = df_2['real_volume'].values.ravel()[currenct_index]
        currenct_index = currenct_index + 1
        final_index = currenct_index

        currenct_vol = currenct_vol + line_vol

      if(start_index == final_index):
        final_index = final_index + 1

      if(final_index < start_index):
        final_index = start_index + 1

      dados_grupo  = df_2[start_index:final_index]

      final_data_  = group_bar(cj = dados_grupo, final = final_index, target_vol= target_vol)
      dados_agrupados.append(final_data_)

      conc = (final_index/len(df_2)) * 100
      conc = round(conc, 2)

      print(str(conc) + "% concluído " + str(len(dados_agrupados)) + " barras até agora")

    return(dados_agrupados)

In [11]:
#Retorna Médias Móveis da variaveis fornecida
def prepare_avg(df = None, time = 20, var = 'close', incs = [5, 15, 30]):
  avg_name = 'avg - ' + str(time) + ' - ' + var
  
  std_name = 'std - ' + str(time) + ' - ' + var

  cv_name  = 'cv - ' + str(time) + ' - ' + var

  bs_name  = 'BS - ' + str(time) + ' - ' + var
  bi_name  = 'BI - ' + str(time) + ' - ' + var

  df[avg_name] = df[var].rolling(time).mean()
  df[std_name] = df[var].rolling(time).std()
  df[cv_name]  = df[std_name]/df[avg_name]

  df[bs_name]  = df[avg_name] + (2 * df[std_name])
  df[bi_name]  = df[avg_name] - (2 * df[std_name])

  for i in range(len(incs)):
    inc = incs[i]

    avg_name_ = avg_name + '_Inc' + str(inc)

    df[avg_name_] = (df[avg_name]/df[avg_name].shift(inc)) - 1

  return(df.copy())

In [12]:
#Requista dados da corretora e agrupa com base na regra estabelecida
def get_ohlc(ativo, timeframe, n=5, volume = 'real_volume'):
    ativo = mt5.copy_rates_from_pos(ativo,timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit='s')
    ativo['time'] = ativo['time'].apply(str)
    times = ativo['time'].values.ravel()
    dates = [select_dates(i) for i in times]
    
    ativo = pd.concat((pd.DataFrame(dates, columns = ['day','year', 'month','hour']), ativo), axis = 1)
    
    ativo_ = fix_data(df_2 = ativo.copy())
    nomes = ativo_.columns
    
    ativo_ = ativo_.values
    ativo_ = pd.DataFrame(ativo_, columns = nomes)
    
    ativo_ = ativo_[3:].reset_index().iloc[:,1:]
    
    ativo_['index'] = ativo_.index

    ativo_final = set_base_var(df = ativo_.copy())
    
    
    dados_agrupados = prepare_group(df_2 = ativo_final.copy())
    
    dados_ = pd.DataFrame(dados_agrupados)
    
    dados_.columns = ['open_time','close_time','open','high', 'low','close', 'vwap', 'volume','vol_dir', 'target_vol','barras','final']
    dados_['Retorno']    = np.log(dados_['close']/dados_['close'].shift())
    dados_['OBV']        = dados_['vol_dir'].cumsum()

    dados_               = prepare_avg(df = dados_.copy(), time = 5, var = 'Retorno', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 20, var = 'Retorno', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 50, var = 'Retorno', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 100, var = 'Retorno', incs = [5, 15, 30])

    dados_               = prepare_avg(df = dados_.copy(), time = 5, var = 'close', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 20, var = 'close', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 50, var = 'close', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 100, var = 'close', incs = [5, 15, 30])

    dados_               = prepare_avg(df = dados_.copy(), time = 5, var = 'OBV', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 20, var = 'OBV', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 50, var = 'OBV', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 100, var = 'OBV', incs = [5, 15, 30])
    
    
    dados_               = prepare_avg(df = dados_.copy(), time = 5, var = 'vwap', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 20, var = 'vwap', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 50, var = 'vwap', incs = [5, 15, 30])
    dados_               = prepare_avg(df = dados_.copy(), time = 100, var = 'vwap', incs = [5, 15, 30])

    dados_ = dados_.dropna().reset_index().iloc[:,1:]
    
    dados_['barra_concluida'] = np.where(dados_['volume'] > dados_['target_vol'],1,0)
    
    return([ativo_final, dados_])

In [13]:
dados   = get_ohlc(ativo,mt5.TIMEFRAME_M5, n = 200000)

0.0% concluído 1 barras até agora
0.01% concluído 2 barras até agora
0.02% concluído 3 barras até agora
0.04% concluído 4 barras até agora
0.05% concluído 5 barras até agora
0.06% concluído 6 barras até agora
0.08% concluído 7 barras até agora
0.09% concluído 8 barras até agora
0.09% concluído 9 barras até agora
0.11% concluído 10 barras até agora
0.12% concluído 11 barras até agora
0.13% concluído 12 barras até agora
0.13% concluído 13 barras até agora
0.15% concluído 14 barras até agora
0.17% concluído 15 barras até agora
0.17% concluído 16 barras até agora
0.18% concluído 17 barras até agora
0.2% concluído 18 barras até agora
0.21% concluído 19 barras até agora
0.25% concluído 20 barras até agora
0.28% concluído 21 barras até agora
0.3% concluído 22 barras até agora
0.34% concluído 23 barras até agora
0.34% concluído 24 barras até agora
0.35% concluído 25 barras até agora
0.37% concluído 26 barras até agora
0.38% concluído 27 barras até agora
0.38% concluído 28 barras até agora
0.4%

3.19% concluído 236 barras até agora
3.2% concluído 237 barras até agora
3.22% concluído 238 barras até agora
3.25% concluído 239 barras até agora
3.26% concluído 240 barras até agora
3.27% concluído 241 barras até agora
3.28% concluído 242 barras até agora
3.29% concluído 243 barras até agora
3.3% concluído 244 barras até agora
3.31% concluído 245 barras até agora
3.34% concluído 246 barras até agora
3.35% concluído 247 barras até agora
3.38% concluído 248 barras até agora
3.39% concluído 249 barras até agora
3.41% concluído 250 barras até agora
3.42% concluído 251 barras até agora
3.44% concluído 252 barras até agora
3.45% concluído 253 barras até agora
3.47% concluído 254 barras até agora
3.48% concluído 255 barras até agora
3.48% concluído 256 barras até agora
3.49% concluído 257 barras até agora
3.5% concluído 258 barras até agora
3.51% concluído 259 barras até agora
3.52% concluído 260 barras até agora
3.53% concluído 261 barras até agora
3.55% concluído 262 barras até agora
3.56

6.79% concluído 494 barras até agora
6.79% concluído 495 barras até agora
6.8% concluído 496 barras até agora
6.81% concluído 497 barras até agora
6.82% concluído 498 barras até agora
6.84% concluído 499 barras até agora
6.84% concluído 500 barras até agora
6.86% concluído 501 barras até agora
6.87% concluído 502 barras até agora
6.88% concluído 503 barras até agora
6.89% concluído 504 barras até agora
6.9% concluído 505 barras até agora
6.92% concluído 506 barras até agora
6.92% concluído 507 barras até agora
6.93% concluído 508 barras até agora
6.94% concluído 509 barras até agora
6.96% concluído 510 barras até agora
6.98% concluído 511 barras até agora
7.0% concluído 512 barras até agora
7.01% concluído 513 barras até agora
7.02% concluído 514 barras até agora
7.05% concluído 515 barras até agora
7.07% concluído 516 barras até agora
7.09% concluído 517 barras até agora
7.09% concluído 518 barras até agora
7.1% concluído 519 barras até agora
7.1% concluído 520 barras até agora
7.11% 

10.19% concluído 752 barras até agora
10.23% concluído 753 barras até agora
10.23% concluído 754 barras até agora
10.25% concluído 755 barras até agora
10.26% concluído 756 barras até agora
10.27% concluído 757 barras até agora
10.29% concluído 758 barras até agora
10.31% concluído 759 barras até agora
10.33% concluído 760 barras até agora
10.35% concluído 761 barras até agora
10.36% concluído 762 barras até agora
10.37% concluído 763 barras até agora
10.38% concluído 764 barras até agora
10.4% concluído 765 barras até agora
10.41% concluído 766 barras até agora
10.42% concluído 767 barras até agora
10.42% concluído 768 barras até agora
10.43% concluído 769 barras até agora
10.44% concluído 770 barras até agora
10.44% concluído 771 barras até agora
10.46% concluído 772 barras até agora
10.48% concluído 773 barras até agora
10.48% concluído 774 barras até agora
10.5% concluído 775 barras até agora
10.52% concluído 776 barras até agora
10.53% concluído 777 barras até agora
10.54% concluí

13.45% concluído 1002 barras até agora
13.46% concluído 1003 barras até agora
13.48% concluído 1004 barras até agora
13.49% concluído 1005 barras até agora
13.51% concluído 1006 barras até agora
13.53% concluído 1007 barras até agora
13.53% concluído 1008 barras até agora
13.56% concluído 1009 barras até agora
13.57% concluído 1010 barras até agora
13.58% concluído 1011 barras até agora
13.61% concluído 1012 barras até agora
13.61% concluído 1013 barras até agora
13.62% concluído 1014 barras até agora
13.63% concluído 1015 barras até agora
13.64% concluído 1016 barras até agora
13.66% concluído 1017 barras até agora
13.68% concluído 1018 barras até agora
13.69% concluído 1019 barras até agora
13.71% concluído 1020 barras até agora
13.72% concluído 1021 barras até agora
13.73% concluído 1022 barras até agora
13.74% concluído 1023 barras até agora
13.74% concluído 1024 barras até agora
13.75% concluído 1025 barras até agora
13.78% concluído 1026 barras até agora
13.8% concluído 1027 barr

...

84.5% concluído 6347 barras até agora
84.52% concluído 6348 barras até agora
84.53% concluído 6349 barras até agora
84.54% concluído 6350 barras até agora
84.55% concluído 6351 barras até agora
84.56% concluído 6352 barras até agora
84.57% concluído 6353 barras até agora
84.58% concluído 6354 barras até agora
84.59% concluído 6355 barras até agora
84.6% concluído 6356 barras até agora
84.61% concluído 6357 barras até agora
84.62% concluído 6358 barras até agora
84.63% concluído 6359 barras até agora
84.64% concluído 6360 barras até agora
84.65% concluído 6361 barras até agora
84.67% concluído 6362 barras até agora
84.68% concluído 6363 barras até agora
84.68% concluído 6364 barras até agora
84.7% concluído 6365 barras até agora
84.71% concluído 6366 barras até agora
84.72% concluído 6367 barras até agora
84.74% concluído 6368 barras até agora
84.76% concluído 6369 barras até agora
84.77% concluído 6370 barras até agora
84.78% concluído 6371 barras até agora
84.79% concluído 6372 barras

87.94% concluído 6597 barras até agora
87.96% concluído 6598 barras até agora
87.97% concluído 6599 barras até agora
87.99% concluído 6600 barras até agora
88.0% concluído 6601 barras até agora
88.02% concluído 6602 barras até agora
88.03% concluído 6603 barras até agora
88.04% concluído 6604 barras até agora
88.06% concluído 6605 barras até agora
88.07% concluído 6606 barras até agora
88.09% concluído 6607 barras até agora
88.11% concluído 6608 barras até agora
88.12% concluído 6609 barras até agora
88.13% concluído 6610 barras até agora
88.16% concluído 6611 barras até agora
88.16% concluído 6612 barras até agora
88.17% concluído 6613 barras até agora
88.18% concluído 6614 barras até agora
88.2% concluído 6615 barras até agora
88.21% concluído 6616 barras até agora
88.22% concluído 6617 barras até agora
88.23% concluído 6618 barras até agora
88.24% concluído 6619 barras até agora
88.26% concluído 6620 barras até agora
88.26% concluído 6621 barras até agora
88.27% concluído 6622 barra

91.43% concluído 6859 barras até agora
91.44% concluído 6860 barras até agora
91.46% concluído 6861 barras até agora
91.49% concluído 6862 barras até agora
91.5% concluído 6863 barras até agora
91.51% concluído 6864 barras até agora
91.53% concluído 6865 barras até agora
91.54% concluído 6866 barras até agora
91.56% concluído 6867 barras até agora
91.58% concluído 6868 barras até agora
91.59% concluído 6869 barras até agora
91.6% concluído 6870 barras até agora
91.62% concluído 6871 barras até agora
91.64% concluído 6872 barras até agora
91.64% concluído 6873 barras até agora
91.65% concluído 6874 barras até agora
91.67% concluído 6875 barras até agora
91.68% concluído 6876 barras até agora
91.69% concluído 6877 barras até agora
91.7% concluído 6878 barras até agora
91.71% concluído 6879 barras até agora
91.73% concluído 6880 barras até agora
91.74% concluído 6881 barras até agora
91.76% concluído 6882 barras até agora
91.77% concluído 6883 barras até agora
91.79% concluído 6884 barras

95.18% concluído 7138 barras até agora
95.2% concluído 7139 barras até agora
95.21% concluído 7140 barras até agora
95.24% concluído 7141 barras até agora
95.25% concluído 7142 barras até agora
95.27% concluído 7143 barras até agora
95.29% concluído 7144 barras até agora
95.29% concluído 7145 barras até agora
95.3% concluído 7146 barras até agora
95.31% concluído 7147 barras até agora
95.32% concluído 7148 barras até agora
95.33% concluído 7149 barras até agora
95.33% concluído 7150 barras até agora
95.34% concluído 7151 barras até agora
95.35% concluído 7152 barras até agora
95.37% concluído 7153 barras até agora
95.37% concluído 7154 barras até agora
95.38% concluído 7155 barras até agora
95.4% concluído 7156 barras até agora
95.42% concluído 7157 barras até agora
95.42% concluído 7158 barras até agora
95.44% concluído 7159 barras até agora
95.45% concluído 7160 barras até agora
95.46% concluído 7161 barras até agora
95.47% concluído 7162 barras até agora
95.49% concluído 7163 barras

99.15% concluído 7415 barras até agora
99.17% concluído 7416 barras até agora
99.18% concluído 7417 barras até agora
99.19% concluído 7418 barras até agora
99.2% concluído 7419 barras até agora
99.21% concluído 7420 barras até agora
99.23% concluído 7421 barras até agora
99.24% concluído 7422 barras até agora
99.24% concluído 7423 barras até agora
99.25% concluído 7424 barras até agora
99.26% concluído 7425 barras até agora
99.28% concluído 7426 barras até agora
99.28% concluído 7427 barras até agora
99.3% concluído 7428 barras até agora
99.32% concluído 7429 barras até agora
99.33% concluído 7430 barras até agora
99.33% concluído 7431 barras até agora
99.33% concluído 7432 barras até agora
99.35% concluído 7433 barras até agora
99.37% concluído 7434 barras até agora
99.38% concluído 7435 barras até agora
99.39% concluído 7436 barras até agora
99.4% concluído 7437 barras até agora
99.41% concluído 7438 barras até agora
99.43% concluído 7439 barras até agora
99.43% concluído 7440 barras

In [14]:
#Dados Originais
d1 = dados[0]

In [15]:
#Dados Agrupados
d2 = dados[1]

In [16]:
d1.head()

,day,year,month,hour,time,open,high,low,close,tick_volume,spread,real_volume,hour_2,volume_m20,Previous_avg,index,vol_dir,Preco_Vol
0,2012-10-18,2012,10,16,2012-10-18 16:15:00,58.54,58.55,58.51,58.51,18,0,55640,2012-10-18_4,416508,387877,0,-55640,3.2555e+06
1,2012-10-18,2012,10,16,2012-10-18 16:20:00,58.51,58.64,58.5,58.64,53,0,73110,2012-10-18_4,416508,387877,1,73110,4.28717e+06
2,2012-10-18,2012,10,16,2012-10-18 16:25:00,58.66,58.66,58.6,58.61,17,0,30820,2012-10-18_4,416508,387877,2,-30820,1.80636e+06
3,2012-10-18,2012,10,16,2012-10-18 16:30:00,58.65,58.71,58.65,58.68,21,0,58980,2012-10-18_4,416508,387877,3,58980,3.46095e+06
4,2012-10-18,2012,10,16,2012-10-18 16:35:00,58.65,58.71,58.63,58.7,46,0,35230,2012-10-18_4,416508,387877,4,35230,2.068e+06


In [17]:
d1.tail()

,day,year,month,hour,time,open,high,low,close,tick_volume,spread,real_volume,hour_2,volume_m20,Previous_avg,index,vol_dir,Preco_Vol
199573,2022-02-14,2022,02,17,2022-02-14 17:35:00,109.58,109.6,109.51,109.54,716,1,60031,2022-02-14_4,1.87725e+06,1.85499e+06,199573,-60031,6.5758e+06
199574,2022-02-14,2022,02,17,2022-02-14 17:40:00,109.53,109.56,109.49,109.56,551,1,21489,2022-02-14_4,1.87725e+06,1.85499e+06,199574,21489,2.35433e+06
199575,2022-02-14,2022,02,17,2022-02-14 17:45:00,109.56,109.67,109.46,109.55,788,1,50945,2022-02-14_4,1.87725e+06,1.85499e+06,199575,-50945,5.58102e+06
199576,2022-02-14,2022,02,17,2022-02-14 17:50:00,109.55,109.61,109.47,109.57,364,1,47498,2022-02-14_4,1.87725e+06,1.85499e+06,199576,47498,5.20436e+06
199577,2022-02-14,2022,02,18,2022-02-14 18:15:00,109.52,109.52,109.52,109.52,97,0,1326301,2022-02-14_4,1.87725e+06,1.85499e+06,199577,0,1.45256e+08
